In [2]:
1+1

2

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Ex1").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/28 23:19:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=Ex1>

In [6]:
R1 = sc.textFile("DataSets/real_estate_sample.txt")
R1.collect()

['Property_ID|Location|Price|Bedrooms|Bathrooms|Size|Price_SQ_FT|Status',
 '1461262|Arroyo Grande|795000|3|3|2371|365.3|Short Sale',
 '1478004|Paulo Pablo|399000|4|3|2818|163.59|Short Sale',
 '1486551|Paulo Pablo|545000|4|3|3032|179.75|Short Sale',
 '1492832|Santa Bay|909000|4|4|3540|286.78|Short Sale',
 '1499102|Thomas Country|109900|3|1|1249|98.99|Short Sale',
 '1489132|Thomas Country|109000|2|1|1129|93.99|Short Sale',
 '1467262|Fort Worth|987000|4|3|2771|465.3|Short Sale',
 '1478114|Paulo Pablo|409000|4|3|2918|223.19|Short Sale',
 '1402551|Nashville|545000|4|3|2932|169.75|Short Sale',
 '1405832|San Jose|980000|4|4|3340|290.98|Short Sale',
 '1493302|Fort Worth|119900|3|2|2249|198.99|Short Sale',
 '1412332|Thomas Country|129000|3|2|1329|73.99|Short Sale',
 '1469062|Arroyo Grande|798000|3|4|2321|235.9|Short Sale',
 '1498004|Nashville|789000|4|3|2419|263.59|Short Sale',
 '1586751|Nashville|896000|4|3|3132|199.75|Short Sale',
 '1433232|Glendale|987000|4|4|3340|216.78|Short Sale',
 '14955

In [7]:
R1.count()

19

In [8]:
R1.first()

'Property_ID|Location|Price|Bedrooms|Bathrooms|Size|Price_SQ_FT|Status'

In [33]:
header = R1.first()
header_cols = header.split("|")
header_cols

['Property_ID',
 'Location',
 'Price',
 'Bedrooms',
 'Bathrooms',
 'Size',
 'Price_SQ_FT',
 'Status']

In [40]:
header_cols_rdd = R1.filter(lambda x:  x.startswith('Property_ID|'))
header_cols_rdd.collect()

['Property_ID|Location|Price|Bedrooms|Bathrooms|Size|Price_SQ_FT|Status']

In [26]:
R2 = R1.filter(lambda x: not x.startswith('Property_ID|'))
R2.collect()

['1461262|Arroyo Grande|795000|3|3|2371|365.3|Short Sale',
 '1478004|Paulo Pablo|399000|4|3|2818|163.59|Short Sale',
 '1486551|Paulo Pablo|545000|4|3|3032|179.75|Short Sale',
 '1492832|Santa Bay|909000|4|4|3540|286.78|Short Sale',
 '1499102|Thomas Country|109900|3|1|1249|98.99|Short Sale',
 '1489132|Thomas Country|109000|2|1|1129|93.99|Short Sale',
 '1467262|Fort Worth|987000|4|3|2771|465.3|Short Sale',
 '1478114|Paulo Pablo|409000|4|3|2918|223.19|Short Sale',
 '1402551|Nashville|545000|4|3|2932|169.75|Short Sale',
 '1405832|San Jose|980000|4|4|3340|290.98|Short Sale',
 '1493302|Fort Worth|119900|3|2|2249|198.99|Short Sale',
 '1412332|Thomas Country|129000|3|2|1329|73.99|Short Sale',
 '1469062|Arroyo Grande|798000|3|4|2321|235.9|Short Sale',
 '1498004|Nashville|789000|4|3|2419|263.59|Short Sale',
 '1586751|Nashville|896000|4|3|3132|199.75|Short Sale',
 '1433232|Glendale|987000|4|4|3340|216.78|Short Sale',
 '1495502|Fort Worth|219900|3|2|987|200.99|Short Sale',
 '1489100|San Jose|107200

In [30]:
R3 = R2.map(lambda x: x.split(",")).map(lambda x: x[0].split("|"))
R3.take(3)

[['1461262',
  'Arroyo Grande',
  '795000',
  '3',
  '3',
  '2371',
  '365.3',
  'Short Sale'],
 ['1478004',
  'Paulo Pablo',
  '399000',
  '4',
  '3',
  '2818',
  '163.59',
  'Short Sale'],
 ['1486551',
  'Paulo Pablo',
  '545000',
  '4',
  '3',
  '3032',
  '179.75',
  'Short Sale']]

In [42]:
header_cols_rdd.collect()

['Property_ID|Location|Price|Bedrooms|Bathrooms|Size|Price_SQ_FT|Status']

In [45]:

header_cols = header_cols_rdd.first().split("|")

i1 = header_cols.index("Property_ID")
i2 = header_cols.index("Location")
i3 = header_cols.index("Size")
i4 = header_cols.index("Price_SQ_FT")

i1,i2,i3,i4

(0, 1, 5, 6)

In [56]:
header_cols_final_rdd = header_cols_rdd.map(lambda x: x.split("|")[i1] +"|"+ x.split("|")[i2] + "|FinalPrice")
header_cols_final_rdd.collect()

['Property_ID|Location|FinalPrice']

In [57]:
#Final_Price = (Size * Price_SQ_FT)

def func_final(Size,Price_SQ_FT):
    #Price_SQ_FT = Price_SQ_FT
    #Size  = 
    return str(float(Size) * float(Price_SQ_FT))

In [59]:
R4 = R3.map(lambda x :x[i1]+"|"+x[i2]+"|"+func_final(x[i3],x[i4]))
R4.take(3)

['1461262|Arroyo Grande|866126.3',
 '1478004|Paulo Pablo|460996.62',
 '1486551|Paulo Pablo|545002.0']

In [60]:
R4.collect()

['1461262|Arroyo Grande|866126.3',
 '1478004|Paulo Pablo|460996.62',
 '1486551|Paulo Pablo|545002.0',
 '1492832|Santa Bay|1015201.2',
 '1499102|Thomas Country|123638.51',
 '1489132|Thomas Country|106114.70999999999',
 '1467262|Fort Worth|1289346.3',
 '1478114|Paulo Pablo|651268.42',
 '1402551|Nashville|497707.0',
 '1405832|San Jose|971873.2000000001',
 '1493302|Fort Worth|447528.51',
 '1412332|Thomas Country|98332.70999999999',
 '1469062|Arroyo Grande|547523.9',
 '1498004|Nashville|637624.21',
 '1586751|Nashville|625617.0',
 '1433232|Glendale|724045.2',
 '1495502|Fort Worth|198377.13',
 '1489100|San Jose|62323.10999999999']

In [61]:
Final_rdd = header_cols_final_rdd.union(R4)
Final_rdd.collect()

['Property_ID|Location|FinalPrice',
 '1461262|Arroyo Grande|866126.3',
 '1478004|Paulo Pablo|460996.62',
 '1486551|Paulo Pablo|545002.0',
 '1492832|Santa Bay|1015201.2',
 '1499102|Thomas Country|123638.51',
 '1489132|Thomas Country|106114.70999999999',
 '1467262|Fort Worth|1289346.3',
 '1478114|Paulo Pablo|651268.42',
 '1402551|Nashville|497707.0',
 '1405832|San Jose|971873.2000000001',
 '1493302|Fort Worth|447528.51',
 '1412332|Thomas Country|98332.70999999999',
 '1469062|Arroyo Grande|547523.9',
 '1498004|Nashville|637624.21',
 '1586751|Nashville|625617.0',
 '1433232|Glendale|724045.2',
 '1495502|Fort Worth|198377.13',
 '1489100|San Jose|62323.10999999999']

In [ ]:
sc.